# Blue Brain BioExplorer

![](./images/bioexplorer_banner.png)

## Script to create a visualization of the Spike

In [79]:
from bioexplorer import BioExplorer, Protein, Sugars, Quaternion
be = BioExplorer('localhost:5000')
be.reset()

True

In [80]:
# Resources
resource_folder = '../tests/test_files/'
pdb_folder = resource_folder + 'pdb/'
complex_folder = resource_folder + 'pdb/glycans/complex/'

protein_representation = BioExplorer.REPRESENTATION_ATOMS_AND_STICKS

In [81]:
# Protein
spike = Protein(
    sources=[pdb_folder + '6vyb.pdb'],
    load_hydrogen=False,
    load_non_polymer_chemicals=False)

name = 'Spike'
status = be.add_protein(
    name=name, protein=spike,
    representation=protein_representation)

## Protein information (AA sequences and glycosylation sites)

In [ ]:
be.get_protein_amino_acid_information(assembly_name=name, name=name)

In [ ]:
status = be.set_protein_color_scheme(
    assembly_name=name, name=name,
    color_scheme=be.COLOR_SCHEME_ATOMS,
    palette_name='Set1', palette_size=2)

## Visualization of amino acids sequence

In [ ]:
status = be.set_protein_amino_acid_sequence_as_range(
    assembly_name=name, name=name, amino_acid_range=[100,150])
status = be.set_protein_color_scheme(
    assembly_name=name, name=name,
    color_scheme=be.COLOR_SCHEME_AMINO_ACID_SEQUENCE,
    palette_name='Set1', palette_size=2)

## Visualization of glycosylation sites

In [89]:
status = be.set_protein_color_scheme(
    assembly_name=name, name=name,
    color_scheme=be.COLOR_SCHEME_GLYCOSYLATION_SITE,
    palette_name='Set1', palette_size=2)

## Visualization of functional regions

In [ ]:
# RGB color palette for amino acid indices
grey = [0.5, 0.5, 0.5]
dark_green = [0.0, 0.5, 0.0]
light_green = [0.0, 1.0, 0.0]

region_indices_and_colors = [
    [   1, grey       ], [   5, grey],  [  30, light_green],  [  41, dark_green],
    [  82, light_green], [  84, grey],  [ 353, grey       ],  [ 357, grey      ],
    [ 800, grey       ]]

# Build a palette according to region colors
palette = list()
for index in range(len(region_indices_and_colors)-1):
    for i in range(region_indices_and_colors[index + 1][0] - 
                   region_indices_and_colors[index][0]):
        palette.append(region_indices_and_colors[index][1])

# Apply palette to other chains
status = be.set_protein_color_scheme(
    assembly_name=name, name=name, 
    color_scheme=be.COLOR_SCHEME_CHAINS,
    palette_name='Greys', palette_size=5)

# Apply palette to region color scheme, only for chain #2
status = be.set_protein_color_scheme(
    assembly_name=name, name=name, 
    color_scheme=be.COLOR_SCHEME_REGION, palette=palette,
    chain_ids=[2])

## Add glycans to protein

In [82]:
glycan_folder = pdb_folder + 'glycans/'
complex_paths = [glycan_folder + 'complex/5.pdb', glycan_folder + 'complex/15.pdb',
                 glycan_folder + 'complex/25.pdb',glycan_folder + 'complex/35.pdb']
high_mannose_paths = [glycan_folder + 'high-mannose/1.pdb', 
                      glycan_folder + 'high-mannose/2.pdb',
                      glycan_folder + 'high-mannose/3.pdb',
                      glycan_folder + 'high-mannose/4.pdb']
hybrid_paths = [glycan_folder + 'hybrid/20.pdb']
o_glycan_paths = [glycan_folder + 'o-glycan/1.pdb']

### High mannose

In [83]:
high_mannose_glycans = Sugars(
    assembly_name=name, name=be.NAME_GLYCAN_HIGH_MANNOSE, 
    protein_name=name, source=high_mannose_paths[0], 
    site_indices=[80,141,253,622,728,736,820,1093],
    representation=protein_representation)
status = be.add_glycans(high_mannose_glycans)

### Hybrid

In [84]:
hybrid_glycans = Sugars(
    assembly_name=name, name=be.NAME_GLYCAN_HYBRID, 
    protein_name=name, source=hybrid_paths[0], 
    site_indices=[657+19],
    representation=protein_representation)
status = be.add_glycans(hybrid_glycans)

### Complex

In [85]:
complex_glycans = Sugars(
    assembly_name=name, name=be.NAME_GLYCAN_COMPLEX, 
    protein_name=name, source=complex_paths[0],
    site_indices=[36,93,168,184,301,350,362,635,1117,1153,1177,1192,1213],
    representation=protein_representation)
status = be.add_glycans(complex_glycans)

### O-Glycans

In [86]:
for index in [323, 325]:
    o_glycan_name = name + '_' + be.NAME_GLYCAN_O_GLYCAN + '_' + \
        str(index)
    o_glycan = Sugars(
        assembly_name=name, name=o_glycan_name,
        source=o_glycan_paths[0], protein_name=name, 
        representation=protein_representation, site_indices=[index])
    be.add_sugars(o_glycan)

## Materials

In [88]:
be.apply_default_color_scheme(shading_mode=be.SHADING_MODE_BASIC)

IntProgress(value=0, max=6)

## Snapshot

In [90]:
from braynsmediamaker import MovieMaker
mm = MovieMaker(be.core_api())

In [91]:
mm.create_snapshot(
    path='/home/favreau/6vyb_all_glycans.png',
    size=[512,512], samples_per_pixel=64,
    exportIntermediateFrames=False
)

IntProgress(value=0, description='In progress...')